In [1]:
# Install modules

# !pip install dash
# !pip install jupyter-dash
# !pip install dash-bootstrap-components
# !pip install statsmodels
# !pip install catboost

# Import Modules
import dash
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from catboost import CatBoostClassifier

In [9]:
model = CatBoostClassifier()
model.load_model('model/catboost_model_multi.dump')

def Header2(name, app):
    title = html.H2(name, style={"margin-top": 5})
    return dbc.Row([dbc.Col(title, md=9)])

def Header1(name, app):
    title = html.H1(name, style={"margin-top": 5})
    return dbc.Row([dbc.Col(title, md=9)])

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

#hr_style example
hr_style = {
    'border': '2px', # 선 두께와 색상을 지정합니다.
}

#레이아웃
app.layout = dbc.Container(
    [
        Header1("불량예측 모델을 통한 수익성 개선", app),
        html.Hr(style=hr_style), #hr_style 적용 예시 
        Header2("Predict", app),
        html.Br(),
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(
                            [
                                html.Label('충전실 온도'),
                                dcc.Slider(id='input1', min=68, max=75, value=71, step=0.1, marks={68: '68', 69: '69', 70: '70', 71: '71', 72: '72', 73: '73', 74: '74', 75: '75'})
                            ],
                            md=6,
                            style={'padding': '10px'}
                        ),
                        dbc.Col(
                            [
                                html.Label('실링 온도'),
                                dcc.Slider(id='input2', min=65, max=141, value=101, step=0.1, marks={65: '65', 75: '75', 85: '85', 95: '95', 105: '105', 115: '115', 125: '125', 135: '135', 141: '141'})
                            ],
                            md=6,
                            style={'padding': '10px'}
                        )
                    ]
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            [
                                html.Label('쿠킹 온도'),
                                dcc.Slider(id='input3', min=100, max=177, value=137, step=0.1, marks={100: '100', 110: '110', 120: '120', 130: '130', 140: '140', 150: '150', 160: '160', 170: '170', 177: '177'})
                            ],
                            md=6,
                            style={'padding': '10px'}
                        ),
                        dbc.Col(
                            [
                                html.Label('쿠킹 스팀압력'),
                                dcc.Slider(id='input4', min=22, max=25, value=23.5, step=0.01, marks={22: '22', 22.5: '22.5', 23: '23', 23.5: '23.5', 24: '24', 24.5: '24.5', 25: '25'})
                            ],
                            md=6,
                            style={'padding': '10px'}
                        )
                    ]
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            [
                                html.Label('실링 압력'),
                                dcc.Slider(id='input5', min=192, max=223, value=208, step=0.1, marks={192: '192', 195: '195', 200: '200', 205: '205', 210: '210', 215: '215', 220: '220', 223: '223'})
                            ],
                            md=6,
                            style={'padding': '10px'}
                        ),
                        dbc.Col(
                            [
                                html.Button(id='submit-button', n_clicks=0, children='결과확인', style={'text-align': 'center', 'font-size': '20px'}),                                
                            ],
                            md=6,
                            style={'padding': '20px', 'display': 'flex', 'justify-content': 'center'}
                        )
                    ]
                )
            ],            
        ),
        html.Div(id='output', style={'text-align': 'center', 'font-size': '20px'})
    ],
    fluid=False,
)

# 입력 데이터를 받아 모델로 예측하는 함수
@app.callback(
    dash.dependencies.Output('output', 'children'),
    [dash.dependencies.Input('submit-button', 'n_clicks')],
    [dash.dependencies.State('input1', 'value'),
     dash.dependencies.State('input2', 'value'),
     dash.dependencies.State('input3', 'value'),
     dash.dependencies.State('input4', 'value'),
     dash.dependencies.State('input5', 'value')])
def predict(n_clicks, input1, input2, input3, input4, input5):
    # 입력 데이터 정규화
    input_data = np.array([[input1, input2, input3, input4, input5]])
        
    # 모델 예측
    prediction = model.predict(input_data)
    
    # # 예측 결과 반환
    # if prediction == 0:
    #     return f'충전실 온도: {input1}, 실링 온도: {input2}, 쿠킹 온도: {input3}, 쿠킹 스팀압력: {input4}, 실링 압력: {input5}, 예측 결과: 정상 생산'
    # else:
    #     return f'충전실 온도: {input1}, 실링 온도: {input2}, 쿠킹 온도: {input3}, 쿠킹 스팀압력: {input4}, 실링 압력: {input5}, 예측 결과: 오류 발생-에러코드{prediction}'
    # 예측 결과 반환
    if prediction == 0:
        return html.Div([
            f'충전실 온도: {input1}, 실링 온도: {input2}, 쿠킹 온도: {input3}, 쿠킹 스팀압력: {input4}, 실링 압력: {input5}',
            html.Br(),
            f'예측 결과: 정상 생산'
        ], style={'font-size': '20px', 'text-align': 'center'})
    else:
        return html.Div([
            f'충전실 온도: {input1}, 실링 온도: {input2}, 쿠킹 온도: {input3}, 쿠킹 스팀압력: {input4}, 실링 압력: {input5}',
            html.Br(),
            f'예측 결과: 오류 발생 - 에러코드 {prediction}'
        ], style={'font-size': '20px', 'text-align': 'center'})

if __name__ == '__main__':
    app.run_server(debug=True, host="127.0.0.6")


Dash is running on http://127.0.0.6:8050/

Dash app running on http://127.0.0.6:8050/
